In [1]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json_shot_scraper import flatten_shot, flatten_goal, flatten_complete_pass, flatten_incomplete_pass, flatten_corner
from  player_scraper import flatten_player, flatten_sub
from dataframe_cleaner import (pass_to_shot, corner_to_shot, transpose_coordinates, coord_to_yards, 
                               shot_distance_angle, dummy_columns, drop_own_goals, goal_dummy)

In [6]:
from html_scraper import db
from mongo_to_db import game_to_cleaned_df, create_frame, create_master_df

In [2]:
pd.set_option('display.max_columns', 50)

In [4]:
db.list_collection_names()

['games']

In [19]:
games = db.games.find()

In [20]:
final_df = create_master_df(games)

/Users/david/galvanize/super_liga_xg/mongo_to_db.py:69: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  master_df = pd.concat([attach_to_df, df], axis=0, ignore_index=True)
/Users/david/galvanize/super_liga_xg/dataframe_cleaner.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  input_df['is_goal'] = input_df.loc[:, 'shot_type'].isin([9, 11, 12, 13]).astype(int)


# EDA

In [21]:
final_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1.0
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21.747333,11.242763,0,0,0.0
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1.0
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,20,38.883333,177285.0,22.67,6.22,3.93,19.67,-1.0,-1.0,0.0,16.803407,59.849102,1,0,0.0
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.220122,32.065607,0,0,0.0


In [22]:
final_df.describe()

,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z2,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,is_goal
count,2955.000000,2955.000000,2955.000000,2955.000000,2955.000000,2955.000000,2955.000000,1221.000000,1221.000000,1221.000000,1221.000000,1221.000000,1221.0,1221.0,2945.000000,2955.000000,2955.000000,2955.000000
mean,83417.628088,15.376156,2.049354,0.202876,0.057773,-0.252988,50.018252,85681.678133,20.738591,17.941474,1.218714,0.186478,-1.0,-1.0,0.086927,17.779827,0.275928,0.109983
std,64229.759131,6.624601,3.816875,8.803820,5.448328,0.738580,26.590282,62086.040107,16.754803,11.353598,18.873264,10.908477,0.0,0.0,0.281776,6.458381,31.213869,0.312922
min,51.000000,0.000000,0.000000,-29.060000,-24.520000,-1.000000,0.250000,51.000000,0.000000,0.440000,-30.270000,-28.450000,-1.0,-1.0,0.000000,0.300000,-90.000000,0.000000
25%,30012.000000,10.670000,0.440000,-6.050000,-3.030000,-1.000000,27.616667,37152.000000,6.220000,10.670000,-14.230000,-6.960000,-1.0,-1.0,0.000000,12.441808,-22.223473,0.000000
50%,69857.000000,13.330000,0.890000,0.000000,0.300000,0.100000,50.600000,70512.000000,20.000000,14.220000,1.210000,-0.300000,-1.0,-1.0,0.000000,16.716028,0.000000,0.000000
75%,126644.000000,20.890000,2.220000,6.660000,3.030000,0.380000,73.208333,122008.000000,30.220000,23.560000,18.160000,6.960000,-1.0,-1.0,0.000000,23.315904,22.826916,0.000000
max,220396.000000,40.890000,88.440000,27.540000,26.030000,1.290000,98.850000,220396.000000,88.890000,83.560000,30.270000,28.760000,-1.0,-1.0,1.000000,47.678323,88.149038,1.000000


In [31]:
#goals that were assisted
final_df[(final_df['is_goal'] == 1) & (final_df['assisted_shot'] == 1)].head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1.0
15,448543,8907.0,11.56,0.44,-10.59,2.42,-1,0.08,22337581,9,136,10.916667,68605.0,21.78,15.11,3.03,-9.38,-1.0,-1.0,0.0,15.677426,-42.492487,1,0,1.0
95,448536,111421.0,18.67,0.44,-8.78,-2.12,-1,0.22,22372861,9,5,8.100000,618.0,1.78,11.56,29.66,-3.63,-1.0,-1.0,1.0,20.631464,-25.186343,1,0,1.0
152,448538,114108.0,8.89,0.00,-4.24,-1.21,-1,0.91,22383020,9,129,79.300000,193188.0,8.89,9.33,-11.80,-3.93,-1.0,-1.0,0.0,9.849350,-25.498359,1,0,1.0
153,448538,114108.0,9.33,0.00,3.03,1.21,-1,0.48,22383255,11,129,82.983333,172760.0,25.33,8.44,-27.24,7.26,-1.0,-1.0,0.0,9.809679,17.991663,1,0,1.0


In [32]:
#missed penalty kicks
final_df[final_df['shot_type'] == 44]

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
170,448541,77919.0,11.56,0.44,-0.30,-2.12,-1,1.10,22392267,44,17,15.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.563892,-1.486581,0,1,0.0
332,448561,46612.0,11.56,1.78,-0.61,6.36,-1,0.29,22462959,44,869,45.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.576083,-3.020592,0,1,0.0


In [35]:
#converted penalty kicks
final_df[final_df['shot_type'] == 13].head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1.0
142,448546,58422.0,11.56,0.44,0.30,0.61,-1,0.26,22378506,13,132,94.766667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.563892,1.486581,0,1,1.0
143,448538,20195.0,11.56,0.00,0.30,-3.03,-1,0.10,22379135,13,12,5.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.563892,1.486581,0,1,1.0
223,448537,55267.0,12.00,0.00,-0.30,-3.93,-1,0.41,23680859,13,10,78.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.003749,-1.432096,0,1,1.0
278,448556,163041.0,12.00,1.78,-0.91,-3.93,-1,0.22,22437147,13,129,83.583333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.034455,-4.336630,0,1,1.0
